# Import

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree

from decisionTree import DTRegressor
from sklearn.utils import resample

# Importing and Splitting data

In [20]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()

def dataset(brand):

    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True)

    for i in ['year']:
        q75,q25 = np.percentile(file.loc[:,i],[75,25])
        IQR = q75-q25
    
        max = q75+(1.5*IQR)
        min = q25-(1.5*IQR)
    
        file.loc[file[i] < min, i] = np.nan
        file.loc[file[i] > max, i] = np.nan

    file = file.dropna(axis = 0)


    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])

    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])

    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(5000)
    # X = file.drop(columns = ['price'])
    # Y = file.price
    # print(file)

    X = file.drop(['price'], axis = 1).to_numpy()
    # X = file[['year', 'mileage', 'tax', 'mpg', 'engineSize']].to_numpy(dtype = 'int')
    Y = file['price'].values.reshape(-1,1)

#     print(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    return  X_train, X_test, Y_train, Y_test

In [21]:
X_train, X_test, Y_train, Y_test = dataset("UKUsedCarDataSet/audi.csv") # Use Audi dataset as default for KNN analysis

# Using Scikit-Learn

In [2]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(random_state = 601)
regressor.fit(X_train, Y_train)
print("R Squared: ", regressor.score(X_test, Y_test))

NameError: name 'X_train' is not defined

In [193]:
from sklearn.model_selection import RandomizedSearchCV
random_grid = params = {'min_samples_split': list(range(2, 100)), 'max_depth': list(range(2, 100)), 'n_estimators': list(range(2, 100))}

#{'min_samples_split': [3, 4, 5, 6], 'max_depth': list(range(1, 6)), 'n_estimators': list(range(1, 100))}

rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=601, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=601),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, 31, ...],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10, 11, 12,
                                                              13, 14, 15, 16,
                                                              17, 18, 19, 20,
                                                              21, 22, 23, 24,
                                                              25, 26, 27, 28,
                      

In [194]:
rf_random.best_params_

{'n_estimators': 25, 'min_samples_split': 2, 'max_depth': 99}

In [15]:
regressor2 = RandomForestRegressor(random_state = 601, n_estimators = 31, min_samples_split = 6, max_depth = 90)
regressor2.fit(X_train, Y_train)
print("R Squared: ", regressor2.score(X_test, Y_test))

#22,6,96 = 0.879540341188823
#25,6,99 = 0.8833043912108435
#31,6,90 = 0.8853793922050485

C:\Users\rb22\AppData\Local\Temp\ipykernel_16076\1224364818.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor2.fit(X_train, Y_train)


R Squared:  0.8853793922050485


In [195]:
from sklearn.model_selection import GridSearchCV
params = {'min_samples_split': list(range(2, 10)), 'max_depth': list(range(90, 100)), 'n_estimators': list(range(10, 50))}

grid_search_cv = GridSearchCV(regressor, params, cv=3)
grid_search_cv.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=601),
             param_grid={'max_depth': [90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18,
                                          19, 20, 21, 22, 23, 24, 25, 26, 27,
                                          28, 29, 30, 31, 32, 33, 34, 35, 36,
                                          37, 38, 39, ...]})

In [203]:
grid_search_cv.best_estimator_

RandomForestRegressor(max_depth=90, n_estimators=31, random_state=601)

In [11]:
regressor3 = RandomForestRegressor(random_state = 601, n_estimators = 23, min_samples_split = 3, max_depth = 5)
regressor3.fit(X_train, Y_train) #14, 5, 5
print("R Squared: ", regressor3.score(X_test, Y_test))
#23, 3, 5 = 0.8554524402174026

R Squared:  0.8554524402174026


# MyCODE

In [57]:
class forestRegression():

    def __init__(self, numTrees = 31, minSample = 6, maxDepth = 90, random_state = 0):
        self.numTrees = numTrees
        self.minSamples = minSample
        self.maxDepth = maxDepth
        self.decisionTree = []
        self.random_state = random_state
        
    # @staticmethod
    def _sample(self, X, y, state):
        sampleNumb, featuresNumb = X.shape
        samples = np.random.RandomState(state).choice(a = sampleNumb, size = sampleNumb, replace = True)
        # samples =rnd.choice(a = n_rows, size = n_rows, replace = True)
        return X[samples], y[samples]
        
    def fit(self, X, y):
        if len(self.decisionTree) > 0:
            self.decisionTree= []
        print("HELLO")
        num_built = 0
        # while num_built < self.numTrees:
        for i in range(self.numTrees):
            print("NUMBER BUILT: ", num_built)
            try:
                clf = DTRegressor(minSamples = self.minSamples, maxDepth = self.maxDepth) ##try 3, then 1
#                 print("GOT HEREE")
                inti = i
                _X, _y = self._sample(X, y, self.random_state + i)
                # _X, _y = resample(X, y, random_state = np.random.RandomState(self.random_state + i))
            
#                 print("sample X: ", _X)
#                 print("sample y: ", _y)
                clf.fit(_X, _y)
#                 print("GOT HEREE 2")
                self.decisionTree.append(clf)
#                 print("GOT HEREE 3")
                num_built += 1
            except Exception as e:
                print("ERROR: ", e)
                continue
    
    def predict(self, X):
        y = []
        for tree in self.decisionTree:
            y.append(tree.predict(X))
        y = np.swapaxes(a = y, axis1 = 0, axis2 = 1) 
        predictions = []
        for preds in y:
            predictions.append(np.mean(preds))
        return predictions

In [ ]:
def userInput():
        chooseBrand = input("Choose your car brand: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen \n")
    
        if chooseBrand == "Audi":
            return "UKUsedCarDataSet/audi.csv"
        elif chooseBrand == "BMW":
            return "UKUsedCarDataSet/bmw.csv"
        elif chooseBrand == "Ford":
            return "UKUsedCarDataSet/ford.csv"
        elif chooseBrand == "Hyundai":
            return "UKUsedCarDataSet/hyundi.csv"
        elif chooseBrand == "Mercedes":
            return "UKUsedCarDataSet/merc.csv"
        elif chooseBrand == "Skoda":
            return "UKUsedCarDataSet/skoda.csv"
        elif chooseBrand == "Toyota":
            return "UKUsedCarDataSet/toyota.csv"
        elif chooseBrand == "Vauxhall":
            return "UKUsedCarDataSet/vauxhall.csv"
        elif chooseBrand == "Volkswagen":
            return "UKUsedCarDataSet/vw.csv"
        else:
            print("Invalid Car Brand")
            return
        
X_train, X_test, Y_train, Y_test = dataset(userInput())
print("\n ***Training Tree Model***")
myForest = forestRegression()  
myForest.fit(X_train, Y_train)

inputPred = []
entries = []

inputPred.append((modelEncoder.transform([input("\nWhat Model is your car? ")]))[0])
inputPred.append(int(input("What year is your car? ")))
inputPred.append((transmissionEncoder.transform([input("What transmission is your car? ")]))[0])
inputPred.append(int(input("How much mileage does your car have? ")))
inputPred.append((fuelTypeEncoder.transform([input("What's your car fuel type? ")]))[0])
inputPred.append(int(input("How much is your cars tax? ")))
inputPred.append(float(input("What's MPG of your car? ")))
inputPred.append(float(input("What the engine size of your car? ")))
entries.append(inputPred)

import time
print("\n ***Predicting***")
start = time.time()
y_pred = myForest.predict([inputPred])
# {0:.2f}'.format()
print("\n Predicted price for your car is: £", y_pred[0])

print("\n ***Predicted in", time.time() - start,"seconds***")

# RS6,2016,Semi-Auto,49050,Petrol,325,29.4,4.0 -- Price is £44,985
# BMW,5 Series,2019,Semi-Auto,4405,Petrol,145,48.7,2.0     Price = £26,000


In [53]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))

In [58]:
myForest = forestRegression()  
myForest.fit(X_train, Y_train)

HELLO
NUMBER BUILT:  0
NUMBER BUILT:  1
NUMBER BUILT:  2
NUMBER BUILT:  3
NUMBER BUILT:  4
NUMBER BUILT:  5
NUMBER BUILT:  6
NUMBER BUILT:  7
NUMBER BUILT:  8
NUMBER BUILT:  9
NUMBER BUILT:  10
NUMBER BUILT:  11
NUMBER BUILT:  12
NUMBER BUILT:  13
NUMBER BUILT:  14
NUMBER BUILT:  15
NUMBER BUILT:  16
NUMBER BUILT:  17
NUMBER BUILT:  18
NUMBER BUILT:  19
NUMBER BUILT:  20
NUMBER BUILT:  21
NUMBER BUILT:  22
NUMBER BUILT:  23
NUMBER BUILT:  24
NUMBER BUILT:  25
NUMBER BUILT:  26
NUMBER BUILT:  27
NUMBER BUILT:  28
NUMBER BUILT:  29
NUMBER BUILT:  30


In [59]:
def evaluation(train, test, yTrain, yTest):    
    y_pred = myForest.predict(test)
    error = rmse(yTest, y_pred) 
    print('The RMSE value is:', error)
    # for i in range(len(y_pred)):
    #     print("\nOriginal value:", yTest[i], "vs the predicted value:", y_pred[i])
    #     print("The difference is:", yTest[i] - y_pred[i])


    file2 = pd.read_csv("UKUsedCarDataSet/audi.csv", quotechar='"', skipinitialspace=True)
    mean= np.mean(file2["price"])

    print("R VALUE:", 1 - (error/mean))



evaluation(X_train, X_test, Y_train, Y_test)

#rmse with 1000: 5771.8260997248035 
#rmse with 5000: 3414.0288745712533
#rmse with resample None: 2431.746126478666
#rmse with resample 601: 3414.0288745712533
#rmse with for loop: 2436.033694659025
#rmse using my method: 2436.033694659025


The RMSE value is: 2436.033694659025
R VALUE: 0.8936075816009895
